<a href="https://colab.research.google.com/github/rafaelsudbrackzimmermann/-MachineLearning/blob/main/cytologia_yolo_destect1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DEBUG = True
DATASET = True
TRAIN = True
INFERENCE = True
SUBMISSION = True
FOLDS = 3
EPOCAS = 50
MODEL_YOLO = 'yolov8s.pt' #'yolov8s.pt'
BATCH = 256
IMGSIZE= 224


# config

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rafaelzimmermann1_cytologia_dataset_path = kagglehub.dataset_download('rafaelzimmermann1/cytologia-dataset')
rafaelzimmermann1_cytologia_images_reprocessadas_path = kagglehub.dataset_download('rafaelzimmermann1/cytologia-images-reprocessadas')
rafaelzimmermann1_yolo_8_best_path = kagglehub.dataset_download('rafaelzimmermann1/yolo-8-best')

print('Data source import complete.')

Resuming download from 294649856 bytes (1108708342 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/rafaelzimmermann1/cytologia-dataset?dataset_version_number=6 (294649856/1403358198) bytes left.


100%|██████████| 1.31G/1.31G [00:35<00:00, 31.3MB/s]

Extracting files...


 10%|▉         | 192M/1.93G [00:07<00:53, 35.2MB/s]

In [ ]:
!pip install ultralytics

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
from PIL import Image
import cv2
import torch
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO

In [ ]:
# # PATH_AUG = '/kaggle/input/cytologia-aug-1/'
# PATH_AUG = '/kaggle/input/cytologia-images-reprocessadas/cytologia-dataset/images/dataset_cytologia'
# PATH_CYTO = '/kaggle/input/cytologia-dataset/'
# PATH_REPROCESS = '/kaggle/input/cytologia-images-reprocessadas/'
# PATH_TEMP = 'kaggle/working/'
# PATH_WORK = 'kaggle/working/'

PATH_AUG = f'{rafaelzimmermann1_cytologia_images_reprocessadas_path}/cytologia-dataset/images/dataset_cytologia'
PATH_CYTO = f'{rafaelzimmermann1_cytologia_dataset_path}/'
PATH_REPROCESS = f'{rafaelzimmermann1_cytologia_images_reprocessadas_path}/'
PATH_BEST = f'{rafaelzimmermann1_yolo_8_best_path}/'
PATH_TEMP = 'content/'
PATH_WORK = 'content/'

In [ ]:
# df = pd.read_csv(f'{PATH_AUG}annotations_augmented.csv')
df = pd.read_csv(f'{PATH_CYTO}cytologia-data-train.csv')
if DEBUG:
    df = df[:1000].copy()
    EPOCAS = 1
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Codificar as classes como números inteiros
label_encoder = LabelEncoder()
df['class_enum'] = label_encoder.fit_transform(df['class'])

# Salve o mapeamento para futuras referências
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Class Mapping:", class_mapping)

# Funçoes

In [ ]:
from PIL import Image

def verify_and_load_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        return np.array(img)
    except Exception as e:
        print(f"Erro ao carregar imagem {image_path}: {e}")
        return None
def resize_to_stride(img, stride=32):
    h, w = img.shape[:2]
    new_h = (h // stride) * stride
    new_w = (w // stride) * stride
    resized_img = cv2.resize(img, (new_w, new_h))
    return resized_img


import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def infer_and_plot(image_path):
    # Verificar e carregar imagem
    img = verify_and_load_image(image_path)
    if img is None:
        return  # Ignorar imagens corrompidas

    # Redimensionar imagem para ser compatível com o modelo
    img = resize_to_stride(img)

    # Converter para tensor e normalizar
    img_tensor = torch.from_numpy(img / 255.0).permute(2, 0, 1).float().unsqueeze(0).to(device)

    # Realizar a inferência
    with torch.no_grad():
        results = model(img_tensor)

    # Verificar se os resultados são uma lista
    if isinstance(results, list):
        results = results[0]  # Pegar o primeiro elemento (primeira imagem)

    # Processar os resultados
    if hasattr(results, "boxes"):
        boxes = results.boxes  # Acessa as caixas
        if boxes is None or len(boxes) == 0:
            print("Nenhuma detecção encontrada.")
            return

        # Iterar sobre cada caixa detectada
        for box in boxes:
            # Coordenadas da caixa
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            conf = box.conf[0].item()  # Confiança
            cls = box.cls[0].item()   # Classe (ID)
            cls_name = results.names[int(cls)]  # Nome da classe

            # Desenhar a caixa na imagem
            box_color = [int(np.random.rand() * 255) for _ in range(3)]
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
            cv2.putText(img, f'{cls_name} {conf:.2f}', (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

    else:
        print("Os resultados não contêm o atributo 'boxes'. Verifique o formato retornado pelo modelo.")
        return

    # Mostrar a imagem
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()



# Cria Dataset

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import os

def copy_images(image_ids, source_folder, destination_folder, max_workers=8):
    """
    Copia imagens de 'source_folder' para 'destination_folder' usando multithreading.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    def copy_image(image_id):
        src = os.path.join(source_folder, image_id)
        dst = os.path.join(destination_folder, image_id)
        shutil.copyfile(src, dst)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(copy_image, image_ids),
                  total=len(image_ids),
                  desc="Copying Images"))

def convert_bbox_to_yolo(image_size, bbox):
    """
    Converte uma bounding box no formato [x1, y1, x2, y2] (valores em pixels)
    para o formato YOLO [x_center, y_center, width, height],
    normalizados entre 0 e 1.
    """
    width_img, height_img = image_size
    x1, y1, x2, y2 = bbox

    w = x2 - x1
    h = y2 - y1
    x_center = x1 + w / 2.0
    y_center = y1 + h / 2.0

    # Normalização
    x_center_norm = x_center / width_img
    y_center_norm = y_center / height_img
    w_norm = w / width_img
    h_norm = h / height_img

    return [x_center_norm, y_center_norm, w_norm, h_norm]

def convert_bbox_to_yolo_vectorized(size, boxes):
    dw, dh = 1. / size[0], 1. / size[1]
    x_min, y_min, x_max, y_max = np.split(np.array(boxes), 4, axis=1)
    x_center = (x_min + x_max) / 2.0 * dw
    y_center = (y_min + y_max) / 2.0 * dh
    width = (x_max - x_min) * dw
    height = (y_max - y_min) * dh
    return np.hstack([x_center, y_center, width, height])

import time

def process_image(image_id, images_folder, labels_folder, df):
    try:
        start = time.time()
        image_path = os.path.join(images_folder, image_id)
        label_path = os.path.join(labels_folder, image_id.split('.')[0] + '.txt')

        # Medir tempo de abertura da imagem
        t1 = time.time()
        img = Image.open(image_path)
        width, height = img.size
        img.close()
        t2 = time.time()

        # Processar labels
        records = df[df['NAME'] == image_id]
        with open(label_path, 'w') as f:
            for _, row in records.iterrows():
                bbox = [row['x1'], row['y1'], row['x2'], row['y2']]
                yolo_bbox = convert_bbox_to_yolo((width, height), bbox)
                yolo_bbox = [round(coord, 6) for coord in yolo_bbox]
                f.write(f"{row['class_enum']} {' '.join(map(str, yolo_bbox))}\n")
        end = time.time()

        # # Debug de tempos
        # print(f"[DEBUG] {image_id} - Total: {end - start:.2f}s | "
        #       f"Abertura: {t2 - t1:.2f}s | Processamento: {end - t2:.2f}s")

    except Exception as e:
        print(f"Erro ao processar {image_id}: {e}")



from concurrent.futures import ThreadPoolExecutor

def create_labels(df, images_folder, labels_folder, max_workers=8):
    """
    Cria arquivos de label no padrão YOLO usando multithreading.
    """
    if not os.path.exists(labels_folder):
        os.makedirs(labels_folder)

    unique_image_ids = df['NAME'].unique()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(
            lambda image_id: process_image(image_id, images_folder, labels_folder, df),
            unique_image_ids),
            total=len(unique_image_ids),
            desc="Creating Labels"
        ))

def process_image(image_id, images_folder, labels_folder, grouped_df):
    try:
        start = time.time()
        image_path = os.path.join(images_folder, image_id)
        label_path = os.path.join(labels_folder, os.path.splitext(image_id)[0] + '.txt')

        # Medir tempo de abertura da imagem
        t1 = time.time()
        with Image.open(image_path) as img:
            width, height = img.size
        t2 = time.time()

        # Processar labels usando o grupo pré-processado
        if image_id in grouped_df:
            records = grouped_df[image_id]
            with open(label_path, 'w') as f:
                for _, row in records.iterrows():
                    bbox = [row['x1'], row['y1'], row['x2'], row['y2']]
                    yolo_bbox = convert_bbox_to_yolo((width, height), bbox)
                    yolo_bbox = [round(coord, 6) for coord in yolo_bbox]
                    f.write(f"{row['class_enum']} {' '.join(map(str, yolo_bbox))}\n")
        end = time.time()

        # Debug de tempos (opcional)
        # print(f"[DEBUG] {image_id} - Total: {end - start:.2f}s | "
        #       f"Abertura: {t2 - t1:.2f}s | Processamento: {end - t2:.2f}s")

    except Exception as e:
        print(f"Erro ao processar {image_id}: {e}")

def create_labels(df, images_folder, labels_folder, max_workers=8):
    """
    Cria arquivos de label no padrão YOLO usando multithreading.
    Otimiza o acesso aos registros agrupando o DataFrame por 'NAME'.
    """
    if not os.path.exists(labels_folder):
        os.makedirs(labels_folder)

    unique_image_ids = df['NAME'].unique()

    # Pré-agrupar o DataFrame por 'NAME' para acesso mais rápido
    grouped = df.groupby('NAME')
    grouped_dict = {name: group for name, group in grouped}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(
            lambda image_id: process_image(image_id, images_folder, labels_folder, grouped_dict),
            unique_image_ids),
            total=len(unique_image_ids),
            desc="Criando Labels"
        ))

# Treino

In [ ]:
# for i in range(3):
#     val_ids = df[df['fold'] == i]['NAME'].unique()
#     train_ids = df[df['fold'] != i]['NAME'].unique()
#     train_df = df[df['NAME'].isin(train_ids)]
#     val_df = df[df['NAME'].isin(val_ids)]


if TRAIN:

    from sklearn.model_selection import StratifiedGroupKFold
    import pandas as pd

    # Adicionar contagem ao DataFrame original
    df['count'] = df.groupby('NAME')['NAME'].transform('count')

    # Obter IDs únicos e suas contagens
    image_ids_counts = df[['NAME', 'count']].drop_duplicates()

    # Inicializar o StratifiedGroupKFold
    sgkf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=42)

    # Criaremos uma lista para armazenar as predições OOF de cada fold
    oof_predictions_list = []

    # Dividir o DataFrame em treino e validação usando StratifiedGroupKFold
    for i, (train_idx, val_idx) in enumerate(sgkf.split(X=image_ids_counts, y=image_ids_counts['count'], groups=image_ids_counts['NAME'])):
        if i+1 > FOLDS:
            break

        train_ids = image_ids_counts.iloc[train_idx]['NAME']
        val_ids = image_ids_counts.iloc[val_idx]['NAME']

        # Filtrar o DataFrame original para criar os conjuntos de treino e validação
        train_df = df[df['NAME'].isin(train_ids)]
        val_df = df[df['NAME'].isin(val_ids)]


        print("#" * 25)
        print(f"### Fold {i+1}")
        display(train_df.shape, val_df.shape)
        display(train_df.head(2), val_df.head(2))
        print("#" * 25)

        # Configurar os caminhos no diretório temporário
        temp_dir = f'/{PATH_TEMP}dataset_{i}'
        images_train_dir = os.path.join(temp_dir, 'images/train')
        images_val_dir = os.path.join(temp_dir, 'images/val')
        labels_train_dir = os.path.join(temp_dir, 'labels/train')
        labels_val_dir = os.path.join(temp_dir, 'labels/val')

        # Criar os diretórios necessários
        os.makedirs(images_train_dir, exist_ok=True)
        os.makedirs(images_val_dir, exist_ok=True)
        os.makedirs(labels_train_dir, exist_ok=True)
        os.makedirs(labels_val_dir, exist_ok=True)

        # Criar o arquivo data.yaml no diretório temporário
        data_yaml = f"""
            train: {images_train_dir}
            val: {images_val_dir}

            nc: 23  # Número de classes no dataset
            names: ['B', 'BA', 'EO', 'Er', 'LAM3', 'LF', 'LGL', 'LH_lyAct', 'LLC', 'LM',
                    'LY', 'LZMG', 'LyB', 'Lysee', 'M', 'MBL', 'MM', 'MO', 'MoB', 'PM',
                    'PNN', 'SS', 'Thromb']  # Lista com os nomes das classes
            """
        yaml_path = f'data_{i}.yaml'
        with open(yaml_path, 'w', encoding='utf-8') as f: # Specify encoding as 'utf-8'
          f.write(data_yaml)

        print(f"data_{i}.yaml criado em {yaml_path}")

        # Copiar imagens de treino
        copy_images(
            image_ids=train_ids,
            # source_folder=f'{PATH_AUG}aug_reprocess_0_5_reba',
            source_folder=f'{PATH_AUG}',
            destination_folder=images_train_dir
        )

        # Copiar imagens de validação
        copy_images(
            image_ids=val_ids,
            # source_folder=f'{PATH_AUG}aug_reprocess_0_5_reba',
            source_folder=f'{PATH_AUG}',
            destination_folder=images_val_dir
        )

        # Criar anotações para o conjunto de treino
        create_labels(
            df=train_df,
            images_folder=images_train_dir,
            labels_folder=labels_train_dir,
            max_workers=4
        )

        # Criar anotações para o conjunto de validação
        create_labels(
            df=val_df,
            images_folder=images_val_dir,
            labels_folder=labels_val_dir,
            max_workers=4
        )


        model = YOLO(MODEL_YOLO)
        model.train(data=f'data_{i}.yaml', epochs=EPOCAS, batch=BATCH, imgsz=IMGSIZE, augment=True)

    #     # --------------------------------------------
    #     # Out-Of-Fold (OOF) Predictions
    #     # --------------------------------------------
    #     # Após treinar, carregamos o "best.pt" do fold i
    #     best_model_path = f'/{PATH_WORK}runs/detect/train{i}/weights/best.pt'
    #     if not os.path.exists(best_model_path):
    #         print(f"Não foi encontrado {best_model_path}. Verifique se o treinamento ocorreu corretamente.")
    #     else:
    #         # Carrega o modelo com os melhores pesos
    #         model_oof = YOLO(best_model_path)
    #         device = 'cuda' if torch.cuda.is_available() else 'cpu'
    #         model_oof.to(device)
    #         model_oof.eval()

    #         # Lista para guardar predições do fold i
    #         fold_predictions = []

    #         # Itera sobre as imagens de validação
    #         for image_id in tqdm(val_ids, desc=f"Inferência OOF Fold {i}", total=len(val_ids)):
    #             img_path = os.path.join(images_val_dir, image_id)
    #             if not os.path.exists(img_path):
    #                 # Se o arquivo não existe no diretório, você pode customizar esse comportamento
    #                 # Por exemplo, talvez as imagens originais estejam em outro lugar.
    #                 # Ajuste conforme a forma que você copiou as imagens.
    #                 continue

    #             # Rodando a inferência
    #             results = model_oof.predict(
    #                         source=img_path,  # Caminho da imagem que será processada.
    #                         device=device,    # Dispositivo a ser usado para inferência ('cuda' ou 'cpu').
    #                         verbose=False,    # Desativa as mensagens detalhadas durante a inferência.
    #                         conf=0.01,         # Confiança mínima para considerar uma detecção (0.0 significa considerar todas).
    #                         iou=0.9,         # Limite de IOU (Intersection Over Union) usado na supressão de caixas redundantes (NMS); valor alto reduz a supressão.
    #                         max_det=10000     # Número máximo de detecções a retornar por imagem; define o limite superior de boxes.
    #                     )

    #             # Extraindo resultados
    #             for result in results:
    #                 boxes  = result.boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
    #                 scores = result.boxes.conf.cpu().numpy()
    #                 labels = result.boxes.cls.cpu().numpy()

    #                 for box, score, label in zip(boxes, scores, labels):
    #                     fold_predictions.append({
    #                         "fold": i,
    #                         "image_id": image_id,
    #                         "x1": float(box[0]),
    #                         "y1": float(box[1]),
    #                         "x2": float(box[2]),
    #                         "y2": float(box[3]),
    #                         "label": int(label),
    #                         "score": float(score)
    #                     })

    #         # Converte para DataFrame e adiciona à lista global de OOF
    #         if fold_predictions:
    #             oof_df = pd.DataFrame(fold_predictions)
    #             oof_predictions_list.append(oof_df)

    # # --------------------------------------------------
    # # Consolidando todas as predições OOF
    # # --------------------------------------------------
    # if oof_predictions_list:
    #     final_oof_df = pd.concat(oof_predictions_list, ignore_index=True)
    #     final_oof_df.to_csv("oof_predictions.csv", index=False)
    #     print("Arquivo oof_predictions.csv salvo com sucesso!")
    #     display(final_oof_df)
    # else:
    #     print("Não foram geradas predições OOF.")

# Inferencia

In [ ]:
if INFERENCE:
    import torch
    import os
    import pandas as pd
    from tqdm import tqdm
    from ultralytics import YOLO

    # Configurações de dispositivo: usar CUDA se disponível
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Carregar o DataFrame de teste
    test_df = pd.read_csv(f'/{PATH_CYTO}cytologia-data-test.csv')

    if DEBUG:
        test_df = test_df[:100]

    # Obter os IDs únicos das imagens de teste
    test_image_ids = test_df['NAME'].unique()

    # Diretório das imagens de teste
    image_dir = f'{PATH_REPROCESS}cytologia-dataset/images/dataset_cytologia/'

    # Lista que vai armazenar o DataFrame de cada fold
    all_fold_predictions = []

    # Defina aqui quais são as pastas de cada fold
    folds = {
        1: "train",
        2: "train2",
        3: "train3"
    }

    for fold_number in range(FOLDS):
        fold_number += 1
        fold_folder = folds[fold_number]
        print(f"== Iniciando inferência do Fold {fold_number} ==")

        # Carregar o modelo do fold correspondente
        model_path = f'/{PATH_WORK}runs/detect/{fold_folder}/weights/best.pt'
        # model_path = f'//kaggle/input/yolo-8-best/YOLO 8v - 50 EPOCAS DIRETO 224px 3 FOLDS/runs/detect/{fold_folder}/weights/best.pt'
        if not os.path.exists(model_path):
            print(f"Modelo não encontrado em {model_path}. Pulando este fold.")
            continue

        model = YOLO(model_path)
        model.to(device)
        model.eval()

        # Lista para armazenar os resultados das previsões deste fold
        prediction_results = []

        # Loop sobre as imagens de teste
        for image_id in tqdm(test_image_ids, desc=f"Inferência Fold {fold_number}", total=len(test_image_ids)):
            # Caminho completo da imagem
            image_path = os.path.join(image_dir, image_id)

            # Verificar se o arquivo existe
            if not os.path.exists(image_path):
                print(f"Imagem {image_path} não encontrada.")
                continue

            # Quantidade “esperada” (ou número de anotações) para essa imagem no df
            num_expected_boxes = len(test_df[test_df['NAME'] == image_id])

            # ------------------
            # 1ª INFERÊNCIA
            # ------------------
            results = model.predict(
                source=image_path,
                device=device,
                verbose=False,
                conf=0.25,
                iou=0.7,
                max_det=300
            )

            # Para simplificar, vamos assumir que só há 1 result por imagem:
            first_result = results[0]
            num_predictions_first = len(first_result.boxes)

            # Se a quantidade de boxes prevista for menor que a necessária,
            # fazemos a 2ª inferência com parâmetros mais flexíveis.
            if num_predictions_first < num_expected_boxes:
                # ------------------
                # 2ª INFERÊNCIA
                # ------------------
                results = model.predict(
                    source=image_path,
                    device=device,
                    verbose=False,
                    conf=0.01,
                    iou=0.9,
                    max_det=10000
                )

            # Processar os resultados (após a inferência que ficou por último)
            final_result = results[0]
            boxes = final_result.boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
            scores = final_result.boxes.conf.cpu().numpy()  # Confidences
            labels = final_result.boxes.cls.cpu().numpy()   # Class labels

            # Adicionar as previsões à lista
            for box, score, label in zip(boxes, scores, labels):
                prediction_results.append({
                    "fold": fold_number,
                    "image_id": image_id,
                    "x1": box[0],
                    "y1": box[1],
                    "x2": box[2],
                    "y2": box[3],
                    "label": int(label),
                    "score": float(score)
                })

        # Converter as previsões para um DataFrame e armazenar
        fold_predictions_df = pd.DataFrame(prediction_results)
        all_fold_predictions.append(fold_predictions_df)

    # Concatenar todos os folds em um único DataFrame
    if all_fold_predictions:
        predictions_df = pd.concat(all_fold_predictions, ignore_index=True)

        # Salvar as previsões para análise posterior
        predictions_df.to_csv("predictions.csv", index=False)

        display(predictions_df)
    else:
        print("Não foram geradas predições em nenhum fold.")


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def visualize_top_images(predictions_df, image_dir, top_n=20):
    """
    Visualiza as N primeiras imagens com suas previsões de bounding boxes.

    Args:
        predictions_df (DataFrame): DataFrame contendo as previsões, com colunas:
            ['image_id', 'x1', 'y1', 'x2', 'y2', 'label', 'score'].
        image_dir (str): Diretório onde as imagens estão armazenadas.
        top_n (int): Número de imagens a serem visualizadas.
    """
    # Obter os IDs das N primeiras imagens
    unique_image_ids = predictions_df['image_id'].unique()[:top_n]

    for image_id in unique_image_ids:
        # Filtrar as previsões para a imagem específica
        image_predictions = predictions_df[predictions_df['image_id'] == image_id]

        # Construir o caminho da imagem
        image_path = f"{image_dir}/{image_id}"

        # Tentar abrir a imagem
        try:
            img = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Imagem {image_path} não encontrada.")
            continue

        # Criar uma figura para exibir a imagem
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(img)

        # Adicionar as bounding boxes e informações às imagens
        for _, row in image_predictions.iterrows():
            x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
            label, score = int(row['label']), row['score']

            # Desenhar a bounding box
            width, height = x2 - x1, y2 - y1
            rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            # Adicionar rótulo e pontuação
            ax.text(
                x1, y1 - 10, f"Label: {label}, Score: {score:.2f}",
                color='red', fontsize=12, backgroundcolor='white'
            )

        plt.title(f"Previsões para {image_id}")
        plt.axis("off")
        plt.show()

# Exemplo de uso:
if INFERENCE:
    visualize_top_images(predictions_df, image_dir=f'{PATH_REPROCESS}cytologia-dataset/images/dataset_cytologia/', top_n=20)


# Submission

In [ ]:
from torchvision.ops import box_iou
if SUBMISSION:

    # Carregar a base de teste
    test_df = pd.read_csv(f'/{PATH_CYTO}cytologia-data-test.csv')

    # Carregar as previsões
    predictions_df = pd.read_csv('predictions.csv')

    # Resultado final
    final_results = []

    # Limiar para evitar sobreposições excessivas
    MAX_IOU = 0.7

    # Iterar por cada imagem na base de teste
    for i, image_id in enumerate(tqdm(test_df['NAME'].unique(), desc="Processando imagens")):
        # IDs esperados para esta imagem
        expected_ids = test_df[test_df['NAME'] == image_id]['trustii_id'].tolist()

        # Previsões para a imagem
        preds = predictions_df[predictions_df['image_id'] == image_id]
        pred_boxes = preds[['x1', 'y1', 'x2', 'y2']].values
        pred_labels = preds['label'].values
        pred_scores = preds['score'].values

        # Ordenar as previsões por pontuação (descendente)
        sorted_indices = np.argsort(-pred_scores)
        pred_boxes = pred_boxes[sorted_indices]
        pred_labels = pred_labels[sorted_indices]
        pred_scores = pred_scores[sorted_indices]

        # print(f'{i} {image_id}')
        # display(f'{pred_boxes} - {pred_scores}')

        # Lista para armazenar as caixas escolhidas (evitar sobreposição)
        selected_boxes = []

        # Associar previsões aos IDs esperados
        for expected_id in expected_ids:
            if len(pred_boxes) > 0:
                for idx, box in enumerate(pred_boxes):
                    # Verificar se a caixa não tem IoU excessivo com caixas já selecionadas
                    if len(selected_boxes) == 0:
                        ious = torch.zeros(1)  # Nenhuma sobreposição na primeira escolha
                    else:
                        ious = box_iou(
                            torch.tensor([box], dtype=torch.float32),
                            torch.tensor(selected_boxes, dtype=torch.float32)
                        )

                    # Se IoU for aceitável, escolha a previsão
                    if ious.max() < MAX_IOU:
                        selected_boxes.append(box)
                        final_results.append({
                            'image_id': image_id,
                            'expected_id': expected_id,
                            'pred_box': box.tolist(),
                            'pred_label': pred_labels[idx],
                            'pred_score': pred_scores[idx]
                        })

                        # Remova a previsão usada
                        pred_boxes = np.delete(pred_boxes, idx, axis=0)
                        pred_labels = np.delete(pred_labels, idx, axis=0)
                        pred_scores = np.delete(pred_scores, idx, axis=0)
                        break
        # if i>10:
        #     break
    # Converter os resultados finais para um DataFrame
    final_results_df = pd.DataFrame(final_results)

    # Salvar os resultados finais
    final_results_df.to_csv('final_results.csv', index=False)

    # Exibir os resultados
    display(final_results_df)

    # ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
if SUBMISSION:
    import os
    import pandas as pd
    from tqdm import tqdm
    import ast  # Para conversão segura de strings para listas

    # Carregar os DataFrames
    test_df = pd.read_csv(f'/{PATH_CYTO}cytologia-data-test.csv')
    predictions_df = pd.read_csv('final_results.csv')

    # Mapear classes numéricas para nomes
    class_mapping = {
        0: "B", 1: "BA", 2: "EO", 3: "Er", 4: "LAM3", 5: "LF", 6: "LGL", 7: "LH_lyAct",
        8: "LLC", 9: "LM", 10: "LY", 11: "LZMG", 12: "LyB", 13: "Lysee", 14: "M",
        15: "MBL", 16: "MM", 17: "MO", 18: "MoB", 19: "PM", 20: "PNN", 21: "SS", 22: "Thromb"
    }

    # Inicializar lista para o DataFrame formatado
    formatted_results = []

    # Iterar pelo DataFrame de teste
    for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        trustii_id = row['trustii_id']
        name = row['NAME']

        # Buscar a previsão correspondente no DataFrame de previsões
        pred = predictions_df[
            (predictions_df['expected_id'] == trustii_id) &
            (predictions_df['image_id'] == name)
        ]

        if not pred.empty:
            # Usar a primeira previsão correspondente
            pred_row = pred.iloc[0]
            try:
                box = ast.literal_eval(pred_row['pred_box'])  # Converter string para lista de forma segura
            except (ValueError, SyntaxError) as e:
                print(f"Erro ao converter pred_box para trustii_id {trustii_id}, NAME {name}: {e}")
                box = [0, 0, 0, 0]  # Valor padrão em caso de erro

            formatted_results.append({
                'trustii_id': trustii_id,
                'NAME': name,
                'x1': int(box[0]),
                'y1': int(box[1]),
                'x2': int(box[2]),
                'y2': int(box[3]),
                'class': class_mapping.get(int(pred_row['pred_label']), "Unknown"),
                'score': float(pred_row['pred_score'])
            })
        else:
            # Se não houver previsão correspondente, verificar previsões apenas pelo 'name'
            pred_0 = predictions_df[predictions_df['image_id'] == name]

            if not pred_0.empty:
                # Usar a primeira previsão correspondente
                pred_row = pred_0.iloc[0]
                try:
                    box = ast.literal_eval(pred_row['pred_box'])  # Converter string para lista de forma segura
                except (ValueError, SyntaxError) as e:
                    print(f"Erro ao converter pred_box para NAME {name}: {e}")
                    box = [0, 0, 0, 0]  # Valor padrão em caso de erro

                formatted_results.append({
                    'trustii_id': trustii_id,
                    'NAME': name,
                    'x1': int(box[0]),
                    'y1': int(box[1]),
                    'x2': int(box[2]),
                    'y2': int(box[3]),
                    'class': class_mapping.get(int(pred_row['pred_label']), "Unknown"),
                    'score': float(pred_row.get('pred_score', 0.0))  # Usar 0.0 se 'pred_score' não existir
                })
            else:
                # Caso não haja previsão correspondente, preencher com valores padrão
                formatted_results.append({
                    'trustii_id': trustii_id,
                    'NAME': name,
                    'x1': 0,
                    'y1': 0,
                    'x2': 0,
                    'y2': 0,
                    'class': "LF",
                    'score': 0.0  # Valor padrão para score
                })

    # Converter para DataFrame final
    final_df = pd.DataFrame(formatted_results)

    # Salvar o DataFrame formatado
    final_df.to_csv('formatted_results.csv', index=False)

    # Exibir o DataFrame final
    display(final_df)


In [ ]:
# final_df['class'] = 'Other1'
if SUBMISSION:
    sub = final_df[['trustii_id',	'NAME',	'x1',	'y1','x2',	'y2',	'class']]

    sub.to_csv('submission.csv', index=False)
    sub

In [ ]:
if SUBMISSION:
    display(sub['class'].value_counts())

In [ ]:
if SUBMISSION:
    final_df_0 = final_df[(final_df['x1'] == 0) & (final_df['y1'] == 0) & (final_df['x2'] == 0) & (final_df['y2'] == 0)]
    display(final_df_0)

In [ ]:
import shutil

# Nome do arquivo ZIP a ser criado no Google Drive
output_filename = '/content/drive/MyDrive/submissao/sub'

# Diretório ou arquivo específico para compactar (ajuste o caminho conforme necessário)
source_path = '/content/formatted_results.csv'  # Exemplo de arquivo típico de submissão

# Compactar o arquivo para um ZIP
shutil.make_archive(output_filename, 'zip', root_dir='/content', base_dir='submission.csv')

print(f'Arquivo compactado criado em: {output_filename}.zip')

In [ ]:
import shutil

# Nome do arquivo ZIP a ser criado no Google Drive
output_filename = '/content/drive/MyDrive/submissao/run'

# Diretório ou arquivo específico para compactar (ajuste o caminho conforme necessário)
source_path = '/content/runs/'  # Exemplo de arquivo típico de submissão

# Compactar o arquivo para um ZIP
shutil.make_archive(output_filename, 'zip', root_dir='/content', base_dir='submission.csv')

print(f'Arquivo compactado criado em: {output_filename}.zip')

In [ ]:
import time

# Set the inactivity time in seconds
inactivity_time = 2 * 60

# Get the current time
start_time = time.time()

while True:
    # Check if the inactivity time has been reached
    if time.time() - start_time > inactivity_time:
        # Log out of Colab
        print("Logging out of Colab...")
        from google.colab import runtime
        runtime.unassign()
        break

    # Sleep for 1 minute
    time.sleep(60)